Feito a partir das aulas do Professor Daniel Hasan - https://www.youtube.com/watch?v=ZkttLNk2jXY&list=PLwIaU1DGYV6skjkahOKtpgs9bPXlrVrIp&index=2&ab_channel=DanielHasanDalip

In [8]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "primeiro_projeto.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [9]:
from app_projeto.models import Tesouro

#Exclui todos os objetos da tabela
Tesouro.objects.all().delete()

# Cria objeto
t1 = Tesouro(nome="Moedas de ouro",quantidade=4,valor=2.4)
# Salva no DB
t1.save()

In [10]:
# Cria e já salva no DB
Tesouro.objects.create(nome="Barril de rum",quantidade=3,valor=343)

<Tesouro: Barril de rum quantidade: 3 R$ 343 >

In [11]:
# Cria e salva vários objetos
Tesouro.objects.bulk_create([\
                Tesouro(nome="Coroa",quantidade=1,valor=23.2),
                Tesouro(nome="Calice",quantidade=3,valor=13.2)])

[<Tesouro: Coroa quantidade: 1 R$ 23.2 >,
 <Tesouro: Calice quantidade: 3 R$ 13.2 >]

In [12]:
# Obter registro do DB
coroa = Tesouro.objects.get(nome="Coroa")
coroa

<Tesouro: Coroa quantidade: 1 R$ 23.20 >

In [13]:
# Obter todos os registros
Tesouro.objects.all()

<QuerySet [<Tesouro: Moedas de ouro quantidade: 4 R$ 2.40 >, <Tesouro: Barril de rum quantidade: 3 R$ 343.00 >, <Tesouro: Coroa quantidade: 1 R$ 23.20 >, <Tesouro: Calice quantidade: 3 R$ 13.20 >]>

In [14]:
# Cria objeto se não existir -> retorna uma tupla
Tesouro.objects.get_or_create(nome="Coroa Linda",quantidade=1,valor=23.2)

(<Tesouro: Coroa Linda quantidade: 1 R$ 23.2 >, True)

In [15]:
# Busca com filtro
Tesouro.objects.filter(nome='Coroa', quantidade=1)

<QuerySet [<Tesouro: Coroa quantidade: 1 R$ 23.20 >]>

In [16]:
Tesouro.objects.filter(nome__startswith="Coroa",quantidade=1)

<QuerySet [<Tesouro: Coroa quantidade: 1 R$ 23.20 >, <Tesouro: Coroa Linda quantidade: 1 R$ 23.20 >]>

In [17]:
# Todos exceto um determinado elemento
Tesouro.objects.exclude(nome__startswith="Coroa")

<QuerySet [<Tesouro: Moedas de ouro quantidade: 4 R$ 2.40 >, <Tesouro: Barril de rum quantidade: 3 R$ 343.00 >, <Tesouro: Calice quantidade: 3 R$ 13.20 >]>

In [18]:
# Concatenado filtros
Tesouro.objects.exclude(nome__startswith="Coroa").filter(valor__lt = 10)

<QuerySet [<Tesouro: Moedas de ouro quantidade: 4 R$ 2.40 >]>

In [19]:
# Limitar as 3 primeiras linhas
Tesouro.objects.all()[:3]

<QuerySet [<Tesouro: Moedas de ouro quantidade: 4 R$ 2.40 >, <Tesouro: Barril de rum quantidade: 3 R$ 343.00 >, <Tesouro: Coroa quantidade: 1 R$ 23.20 >]>

In [21]:
# Filtros com outros campos - Valor menor que a quantidade
from django.db.models import F
Tesouro.objects.filter(valor__lt = F('quantidade'))

<QuerySet [<Tesouro: Moedas de ouro quantidade: 4 R$ 2.40 >]>

In [23]:
# Exibir apenas o nome e a quantidade - Projeção
# values_list retorna em lista ao invés de dicionário
# Usar .distinct() para eliminar elementos repetidos
# Documentação de consultas(queries) - https://docs.djangoproject.com/en/3.0/topics/db/queries/
Tesouro.objects.all().values("id","nome","quantidade")

<QuerySet [{'id': 7, 'nome': 'Moedas de ouro', 'quantidade': 4}, {'id': 8, 'nome': 'Barril de rum', 'quantidade': 3}, {'id': 9, 'nome': 'Coroa', 'quantidade': 1}, {'id': 10, 'nome': 'Calice', 'quantidade': 3}, {'id': 11, 'nome': 'Coroa Linda', 'quantidade': 1}]>

In [29]:
# Campos calculados
# Operações no banco de dados são mais otimizadas - menos consultas mais rápido será a aplicação
# Documentação Django Expressions - https://docs.djangoproject.com/en/3.0/ref/models/expressions/
# Calcular o valor total

from django.db.models import F,ExpressionWrapper,DecimalField

# Define o tipo
tipo_campo_calculado = DecimalField(max_digits=10,decimal_places=2,blank=True)

# Descreve o campo
expressao_valor_total = ExpressionWrapper(F('valor')*F('quantidade'),output_field=tipo_campo_calculado)

# Faz a consulta com o campo calculado
lista = Tesouro.objects.annotate(total=expressao_valor_total)

print(f"Tesouro: {lista[0]} Total: {lista[0].total}")

Tesouro: Moedas de ouro quantidade: 4 R$ 2.40  Total: 9.60000000000000


In [30]:
# Obtém a quantidade total de tesouros
from django.db.models import Sum
Tesouro.objects.aggregate(Sum("quantidade"))

{'quantidade__sum': 12}

In [31]:
# Obtém, para cada nome, a quantidade total
Tesouro.objects.values("nome").annotate(Sum("quantidade"))

<QuerySet [{'nome': 'Barril de rum', 'quantidade__sum': 3}, {'nome': 'Calice', 'quantidade__sum': 3}, {'nome': 'Coroa', 'quantidade__sum': 1}, {'nome': 'Coroa Linda', 'quantidade__sum': 1}, {'nome': 'Moedas de ouro', 'quantidade__sum': 4}]>

In [32]:
# Atualiza o tesouro com nome 'Coroa'
t = Tesouro.objects.get(nome="Coroa")
t.quantidade = 10
t.save()

In [33]:
# Deleta elemento de nome "Coroa"
Tesouro.objects.get(nome="Coroa").delete()

(1, {'app_projeto.Tesouro': 1})

In [35]:
from datetime import datetime 
from app_projeto.models import *
bloguinho = Blog.objects.create(nome="Bloguinho",sobre="Este é um blog")

autores = [Author(nome="Matheus",email="matheus@email.com"),
                                      Author(nome="Rebeca",email="resb@email.com"),
                                     Author(nome="Pepe",email="peps@email.com")]

for autor in autores:
  autor.save()

posts = [Post(blog=bloguinho,titulo="Meu primeiro post",rating=10, texto="la",data_publicacao=datetime.now()),
        Post(blog=bloguinho,titulo="Meu segundo post",rating=10, texto="la2",data_publicacao=datetime.now()),
        Post(blog=bloguinho,titulo="Meu terceiro post",rating=10, texto="la3",data_publicacao=datetime.now()),]

for post in posts:
    post.save()

# Vincula os autores aos posts
posts[0].autores.add(autores[0])
posts[1].autores.add(autores[1])
posts[1].autores.add(autores[2])
posts[2].autores.add(autores[2])

In [37]:
# Exibe o nome do blog e titulo do post
Post.objects.values("blog__nome","titulo")

<QuerySet [{'blog__nome': 'Bloguinho', 'titulo': 'Meu primeiro post'}, {'blog__nome': 'Bloguinho', 'titulo': 'Meu segundo post'}, {'blog__nome': 'Bloguinho', 'titulo': 'Meu terceiro post'}]>

In [40]:
# Contabiliza o numero de posts por autor
from django.db.models import Count
autores = Author.objects.annotate(Count('post'))
autores[0].post__count

0

 - É possível utilizar o Django para acessar uma base de dados legado
 - O Django gera automaticamente as classes para essa base de dados com o comando python manage.py inspectdb > models.py
 - É recomendável criar um aplicação nova para cada base de dados legado
 - https://docs.djangoproject.com/en/3.0/howto/legacy-databases/

Recomendações
 - Minimize o número de consultas ao banco quando possível
 - Use índice quando possível para facilitar consultas repetitivas (ex: busca por CPF, nome da cidade)
- Aprenda a usar cache para deixar as consultas ainda mais rápidas
- SQLLite: recomendável apenas para testes
- Use Transações quando necessário
- Deixe a base de dados normalizada - ou seja, evite redundancia de dados
- Use as classes Django para alterar a estrutura do banco de dados